In [9]:
import numpy as np
import pandas as pd

df = pd.read_csv("/Users/alexyku/Desktop/shapley_values_64_25_gm12878_depth_5_19.txt", delimiter="\t")
df = df.fillna(0)  # NaN values ==> 0.0
df["hypothesis"] = [frozenset(eval(x)) for x  in df["permutation"]]

_counter = {"offline": 0, "online": 0}

def reset_counter():
    # resets the counter dict
    _counter = {"offline": 0, "online": 0}

def evaluate_offline(hypothesis):
    """Uses pre-computed values."""
    _counter["offline"] += 1
    # hypothesis: frozenset of marks
    series = df[df["hypothesis"] == hypothesis
               ]["maskedAverageAuROC"]
    return float(series) if series.size == 1 else 0.0

def evaluate_online(hypothesis):
    """Pass through validation set live."""
    _counter["online"] += 1
    # marks = list(hypothesis)
    # keep_mask = keep_mask_from_marks(cell_type, marks)
    # y_true, y_score = infer_fn(validation_data)
    # return auc(y_true, y_score)
    return 0.0

evaluate = evaluate_offline  # Use offline version.

df.sort_values("maskedAverageAuROC", ascending=False)[["permutation", "maskedAverageAuROC"]].head()

,permutation,maskedAverageAuROC
2103,"['CHD2', 'DNase', 'EZH2', 'RFX5']",0.944609
2173,"['CHD2', 'DNase', 'Pol2', 'RFX5']",0.939678
2158,"['CHD2', 'DNase', 'NRSF', 'RFX5']",0.939620
2139,"['CHD2', 'DNase', 'Max', 'RFX5']",0.939454
2098,"['CHD2', 'DNase', 'EZH2', 'Max']",0.938243


In [41]:
all_features = frozenset(sum([eval(x) for x in df["permutation"]], []))
max_subset_size = 4

num_beams = 4
beams = []  # List of tuples containing: (score, subset) of type (float, frozenset)

null_hypothesis = frozenset()
null_score = evaluate(null_hypothesis)

beams.append((null_score, null_hypothesis))
cache = {null_hypothesis: null_score}

for _ in range(max_subset_size):
    new_beams = []  # Next generation of beams
    for score, beam in beams:
        # Iterate through features that can be added
        for feature in (all_features - beam):
            # A hypothesis extends an existing beam
            # with a single feature
            hypothesis = beam.union({feature})
            # Prevent evaluating the same beam twice
            if hypothesis in cache:
                score = cache[hypothesis]
            else:
                score = evaluate(hypothesis)
                cache[hypothesis] = score
            # Extend the generation of beams
            new_beams.append((score, hypothesis))
    # Keep the best beams from this generation and the next
    # Note: beams can be different lengths though we hope
    # to see a monotonic improvement with more features
    beams = set(new_beams + beams)
    beams = sorted(beams, reverse=True)
    beams = beams[:num_beams]
    
# pd for pretty printing
pd.DataFrame([{"maskedAverageAuROC": k, "permutation": v} for k, v in beams])

,maskedAverageAuROC,permutation
0,0.944609,"(EZH2, RFX5, CHD2, DNase)"
1,0.939678,"(RFX5, Pol2, CHD2, DNase)"
2,0.939620,"(RFX5, CHD2, NRSF, DNase)"
3,0.939454,"(RFX5, Max, CHD2, DNase)"


In [40]:
tbl = pd.DataFrame([{
    "permutation": k,
    "maskedAverageAuROC": v,
    "numMarks": len(k)
}for k, v in cache.items()])

for subset_size in range(max_subset_size + 1):
    print(tbl[tbl["numMarks"] == subset_size].sort_values(
        "maskedAverageAuROC", ascending=False).head(num_beams))
    print("\n")

   maskedAverageAuROC  numMarks permutation
0            0.869723         0          ()


    maskedAverageAuROC  numMarks permutation
5             0.905142         1      (CHD2)
13            0.897919         1     (DNase)
14            0.890875         1      (p300)
19            0.881763         1      (CTCF)


    maskedAverageAuROC  numMarks    permutation
31            0.924376         2  (CHD2, DNase)
29            0.914462         2   (RFX5, CHD2)
24            0.912758         2   (EZH2, CHD2)
49            0.912507         2  (DNase, p300)


     maskedAverageAuROC  numMarks          permutation
95             0.934909         3  (RFX5, CHD2, DNase)
90             0.933359         3  (EZH2, CHD2, DNase)
101            0.928750         3   (Max, CHD2, DNase)
92             0.928732         3  (Pol2, CHD2, DNase)


     maskedAverageAuROC  numMarks                permutation
154            0.944609         4  (EZH2, RFX5, CHD2, DNase)
156            0.939678         4  (RFX5, 